In [43]:
using LinearAlgebra, BenchmarkTools
a = 2.0;
A = randn(100,100)
C1 = zeros(100,100)
C2 = zeros(100,100)
C3 = zeros(100,100)

# Broadcasting (concise)
C1 .+= a .* A;

# Full broadcast (no temporary): equivalent but preferred
@. C2 = a*A + C2          # or: C .+= a .* A

# Loop 
function axpy_loop!(C::AbstractMatrix, a::Real, A::AbstractMatrix)
    @assert size(C) == size(A)
    @inbounds @simd for I in eachindex(C, A)  # single loop var
        C[I] += a * A[I]
    end
    return C
end


axpy_loop!(C3, a, A)
			


100×100 Matrix{Float64}:
  3.06823     0.188482   -0.293598   …  -1.3985      0.873409   -4.80662
  0.267111    1.54964    -0.925203       1.73115    -0.786247   -2.88792
  1.37288     3.07455    -0.297764       2.35248    -0.0468176  -0.347723
 -2.18212    -5.49558     1.19397       -2.69544     4.72384    -0.337328
 -0.599005   -0.0453578  -2.15892        0.444046    1.5836     -0.342661
 -2.21751    -1.28604     1.82187    …   5.95419    -2.91264    -0.209098
  0.155179   -2.7286      1.00927       -0.777033    0.631587    0.563623
  0.276133   -0.676027    0.755417      -4.35764     3.20579    -2.91634
  0.236971   -3.21606    -2.06333       -1.38011    -0.408447   -0.234859
 -1.11607    -0.164926    1.74615       -1.80699     1.47335     0.331788
  ⋮                                  ⋱                          
 -3.00751    -2.29624    -0.934512       1.96319     0.619666    0.56802
  0.0598744  -3.11089     3.81527        0.440807   -0.671516    2.18509
 -1.37614    -4.19853     0

In [47]:
#Goal A: matrix--matrix and matrix--vector products (use BLAS).
A = randn(100,100)
B = randn(100,100)
C = zeros(100,100)
x = randn(100, 1)
# BLAS-backed ops
    Y = A * x                  # gemv (matrix-vector)
C = A * B                  # gemm (matrix-matrix)
# In-place BLAS (avoid allocations)
	mul!(Y, A, x)              # Y := A*x
	mul!(C, A, B, 1.0, 0.0)    # C := 1*A*B + 0*C	

100×100 Matrix{Float64}:
  13.5034      7.16928     2.87133   …    2.93081      4.72342   8.9705
   7.15197    -2.7403    -11.3367       -18.7794      -1.70617   5.32419
  -7.02834   -10.2386    -26.6055        -1.45907     15.204     7.25671
   6.64457     0.565389   -3.28281        5.42363    -10.1185   18.4843
 -18.892      -2.66946    12.0985         0.26103     -6.75699   7.52076
  -6.37011    -9.93973     3.85155   …   -2.0253      15.557     7.18342
 -15.5851     -3.81696   -20.0629        -0.0361116  -18.0541   10.7027
   7.50061    11.9908     -3.57642       -4.42531      5.07363  -8.82656
  -7.62574    -0.662304  -17.0659         2.76536     11.8035    6.60638
   0.440555   16.4297      4.59267       19.0627     -25.0244    8.00847
   ⋮                                 ⋱                          
  -6.86513    11.9854     -0.711867      -6.07634      7.49112   8.5917
  -3.81575     3.06698     6.06319       -7.90796     -7.91984   9.57044
   3.63217    -6.36469    -1.00216    

In [48]:
#Goal B: solve many linear systems with the same $A$ \emph{without} re-factorizing.
	
	# One-off solve (preferred over inv(A)*B)
	X = A \ B
    # Many solves with same A: factor once, reuse
    F = lu(A)                  # or cholesky(A) if SPD
	X  = F \ B                 # multiple RHS columns

100×100 Matrix{Float64}:
 -1.93829       5.07903   -3.93736   …  -1.37105     -3.3834     1.36807
  1.11556      -7.60022    6.04889       1.16396      9.05676   -5.24956
 -3.84585      14.1717   -10.9501       -2.88281    -13.1011     6.8407
 -3.32569      15.2922   -13.5151       -2.85738    -16.8449    10.0033
  3.52274     -14.9336    13.1521        2.69997     15.3676    -8.73781
 -0.459187      3.78857   -3.74241   …  -0.624798    -5.1475     3.21069
  1.55447      -6.44382    6.83127       1.05275      4.72496   -2.04543
 -1.09617       5.86348   -6.63774      -1.21965     -6.23795    3.33333
 -0.0338689     2.86127   -2.99747      -0.60773     -4.55881    2.99004
  3.39538     -15.5887    13.9478        3.71453     14.7219    -8.25025
  ⋮                                  ⋱                         
  2.01542     -11.3384    10.8135        2.23521     12.6922    -7.41928
 -6.39604      22.4265   -22.133        -4.16449    -21.6361    10.7102
 -3.49981      14.167    -12.8992     

In [50]:
using Printf
x = 0.1 + 0.2
@printf("%.17f\n", x)  # 0.30000000000000004

isapprox(x, 0.3; atol=1e-12)

0.30000000000000004


true

In [51]:
using LinearAlgebra

A = [1.0 1.0; 1.0 1.000001]
cond(A)

4.0000020003309543e6

In [53]:


#2.1. Linear Algebra
using LinearAlgebra
A = randn(3,3); b = randn(3)
A*b          # matrix-vector
A*A'         # matrix-matrix



3×3 Matrix{Float64}:
  3.79282  -3.56369    1.19521
 -3.56369   4.55791   -0.522574
  1.19521  -0.522574   3.0904

In [56]:
A = randn(3,3); b = randn(3)
x = A \ b            # solve Ax = b
# If repeated solves with same A, factorize once:
F = lu(A); x2 = F \ b

3-element Vector{Float64}:
  1.4260109169972115
  1.7480026612638873
 -1.8801442514828237

In [60]:
F = qr(randn(4,3))
Q = Matrix(F.Q); R = F.R
S = svd(randn(4,3))
U, s, Vt = S.U, S.S, S.Vt

([0.07754785020366017 -0.4324431486404737 -0.4781583703912006; 0.6108189577272346 0.5625347554030448 -0.55014205546898; -0.6552460808576768 0.002560247645509328 -0.6683965467053052; -0.43765180832536305 0.7046566662492756 0.1481699969009242], [2.69141687923271, 1.9856324857974474, 0.847013258564181], [0.5116838804348208 -0.42488599497535184 -0.7467606696773532; 0.30869384773362235 0.9020393999126886 -0.3017168032718228; 0.8018027905130882 -0.0761367997683231 0.5927187130894916])

In [61]:
A = Symmetric(randn(4,4)); A = A*A'  # SPD
D, V = eigen(A)  # eigen decomposition

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
4-element Vector{Float64}:
 0.020776227690955995
 0.8008911873147708
 1.3591898491101775
 2.2764333257710954
vectors:
4×4 Matrix{Float64}:
  0.364574  -0.309112   0.875917    0.0656116
 -0.902843   0.028787   0.397945   -0.160268
  0.151965  -0.138801  -0.0389895  -0.977814
  0.169903   0.940402   0.269979   -0.11785

In [63]:
using SparseArrays
S = spdiagm(0 => fill(2.0,5), 1 => fill(-1.0,4), -1 => fill(-1.0,4))
b = ones(5)
x = S \ b

5-element Vector{Float64}:
 2.5
 4.000000000000001
 4.500000000000001
 4.000000000000002
 2.5000000000000004

In [64]:
#2.2. Root Finding
# Solve x = cos(x)
g(x) = cos(x)

function fixedpoint(g, x0; tol=1e-12, maxit=10_000)
    x = x0
    for k in 1:maxit
        xn = g(x)
        if abs(xn - x) < tol
            return xn, k
        end
        x = xn
    end
    error("no convergence")
end

x, it = fixedpoint(g, 0.5)
(x, it, cos(x))  # x ≈ 0.739085..., cos(x) ≈ x

(0.739085133215467, 69, 0.7390851332149543)

In [65]:
#Newton method:
# Solve f(x) = x^2 - 2 = 0
	f(x) = x^2 - 2
	fp(x) = 2x
	x = 1.0
	for k in 1:10
	x -= f(x)/fp(x)
	end
	x


1.414213562373095

In [66]:
using LinearAlgebra
	function newton(F, J, x; iters=20)
	for k in 1:iters
	x -= J(x) \ F(x)
	end
	return x
	end

#1) Scalar example: solve x^2 - 2 = 0 (root = √2)
F1(x) = x^2 - 2
J1(x) = 2x
x1 = newton(F1, J1, 1.0)
@show x1  F1(x1)

x1 = 1.414213562373095
F1(x1) = -4.440892098500626e-16


-4.440892098500626e-16

In [67]:
#Optimization
# Gradient descent demo
	f(x) = (x-2)^2
	fp(x) = 2(x-2)
	x = 0.0
	for k in 1:20
	x -= 0.2*fp(x)
	end
x

1.9999268768311989

In [68]:
#Line Search
function linesearch(f, x; d=-1.0, alpha=1.0, beta=0.5)
		fx = f(x)
		while f(x + alpha*d) > fx
		alpha *= beta
		end
		return alpha
		end

linesearch(f, x)
f(x)


5.346997815521184e-9

In [70]:
#quasi-Newton method 
function steepest_descent(f, g, x; steps=100)
			for k in 1:steps
			d = -g(x)
			a = 0.1
			x += a*d
			end
			return x
			end

# f(x,y) = (x-3)^2 + (y+1)^2  -> minimum at (3,-1)
f(v) = (v[1]-3)^2 + (v[2]+1)^2
g(v) = [2(v[1]-3); 2(v[2]+1)]

x0 = [0.0, 0.0]
xmin = steepest_descent(f, g, x0; steps=30)
@show xmin f(xmin)

xmin = [2.996286179882144, -0.9987620599607145]
f(xmin) = 1.5324955408659133e-5


1.5324955408659133e-5

In [72]:
#Numerical Differentiation and Integration
# Forward difference
fd(f, x, h=1e-6) = (f(x+h) - f(x)) / h
fd(sin, 0.3)



0.9553363413683513

In [28]:
#Central difference
cd(f, x, h=1e-6) = (f(x+h) - f(x-h)) / (2h)
cd(exp, 0.0)

0.9999999999732445

In [ ]:
function simpson(f, a, b, n)
			n % 2 == 0 || error("n must be even")
			h = (b-a)/n
			s = f(a) + f(b)
			for k in 1:2:n-1
			s += 4*f(a + k*h)
			end
			for k in 2:2:n-2
			s += 2*f(a + k*h)
			end
			return s*h/3
			end

# Example: ∫_0^1 e^{-x^2} dx

# New function: (−x^2 + 3x), 1000 points, integral: ∫_0^1 (−x^2 + 3x) dx
#f(x) = exp(-x^2)
#val = simpson(f, 0.0, 1.0, 200)  # n must be even

0.7468241328175366

In [74]:
#Interpolation
x = 0:0.5:5
y = @. sin(x) + 0.1*x
function lininterp(xg, yg, x)
	i = searchsortedlast(xg, x)
	i == length(xg) && (i -= 1)
	t = (x - xg[i])/(xg[i+1]-xg[i])
	return (1-t)*yg[i] + t*yg[i+1]
	end


lininterp(collect(x), collect(y), 2.2)

1.0049673137369914

In [ ]:
#use Interpolations
using Interpolations

#create some data
A_x = 1.0:4.0:50.0

A   = collect(2 .* A_x)       

itp  = interpolate(A, BSpline(Cubic(Line())), OnGrid())
sitp = scale(itp, A_x)

sitp(3.0)                     

5.999999999999998

In [78]:
#ODEs Euler approach
	# x' = -x, x(0)=1
			f(t,x) = -x
function euler(f, t0, x0, h, n)
			t = t0; x = x0
	for k in 1:n
		x += h*f(t, x)
		t += h
	end
	return x
end
euler(f, 0.0, 1.0, 0.01, 100)

0.3660323412732296

In [79]:
#Runge-Kutta
function rk4(f, t0, x0, h, n)
			t = t0; x = x0
			for k in 1:n
			k1 = f(t, x)
			k2 = f(t + h/2, x + h*k1/2)
			k3 = f(t + h/2, x + h*k2/2)
			k4 = f(t + h, x + h*k3)
			x += h*(k1 + 2k2 + 2k3 + k4)/6
			t += h
			end
			return x
			end
			rk4(f, 0.0, 1.0, 0.01, 100)

0.3678794412023554

In [82]:
#Randomness and Monte Carlo:
using Random
Random.seed!(2025)
rand()
randn()

0.09251584324213831

In [84]:
# Estimate E[g(Z)] for Z ~ N(0,1)
using Statistics
function mc(g, N=10_000)
		z = randn(N)
	return mean(g.(z))
	end
mc(z->z^2)

1.0054543949220682

In [85]:
function mc_anti(g, N=10_000)
			z = randn(div(N,2))
			return mean((g.(z) .+ g.(-z))/2)
end

mc_anti(z->z^2)

0.9645952586693846

In [21]:
#Performance:
x = rand(1_000_000)
y = similar(x)
y .= @. 2x + 1  # fused broadcast

1000000-element Vector{Float64}:
 1.2360525122140347
 1.5136860567087436
 1.7291358537306267
 1.1048393406018215
 1.9320771134467456
 2.0698893403437815
 2.132714191881435
 1.048841772625953
 1.9413107004709618
 1.2581801921508664
 ⋮
 2.5952016772691757
 2.5169478516802393
 1.7919659272128563
 2.0589330521363784
 2.766654127244422
 1.0803784308287927
 2.7715635483905343
 1.6754415377496916
 1.9017749081788207

In [22]:
using BenchmarkTools
	f(x) = sum(@. 2x + 1)
	x = rand(10_000)
@btime f($x)

  2.844 μs (3 allocations: 78.20 KiB)


19983.150320845925

In [ ]:
#PRACTICE
#Task T.2.1 
	f(x) = x^3 - 2
	fp(x) = 3x^2
	x = 1.0
	for i in 1:20
		x -= f(x)/fp(x)
	end
		x

1.2599210498948732

In [91]:
#Task T.2.2 
    using LinearAlgebra, Random, BenchmarkTools
	Random.seed!(1)
	A = randn(200,200); b = randn(200)
		@btime A \ b;
		@btime inv(A) * b;

  331.900 μs (7 allocations: 315.83 KiB)
  749.400 μs (11 allocations: 415.94 KiB)


In [93]:
#Task T2.3
function simpson(f,a,b,n)
		n % 2 == 0 || error("n must be even")
		h = (b-a)/n
		s = f(a) + f(b)
		for k in 1:2:n-1; s += 4*f(a + k*h); end
		for k in 2:2:n-2; s += 2*f(a + k*h); end
		return s*h/3
		end

simpson(x->(-x^2+3x), 0.0, 1.0, 1000)

1.166666666666667

In [99]:
#Task T2s.1
using LinearAlgebra
del = 1e-15
A = [1.0 1.0 1.0; 1.0 1.0+del 1.0 ; 1.0 1.0 1.0+del]
cond(A)

7.800463371553966e15

In [102]:
b = rand(Float32, 3) 
x1 = A \ b                 # preferred: solves via factorization
x2 = inv(A) * b            # explicitly forms inverse (numerically fragile)

@show norm(x1 - x2)


norm(x1 - x2) = 0.0897587913521567


0.0897587913521567

In [106]:
#Task T2s.2
#Compare (i) broadcast vs (ii) hand loop for $C \leftarrow aA + C$, then compare solves:

using Random, LinearAlgebra, BenchmarkTools
n = 1500
a = 2.0
A  = randn(n,n)
C1 = similar(A); fill!(C1, 0.0)
C2 = similar(A); fill!(C2, 0.0)

# Fused broadcast AXPY: C1 := a*A + C1
@. C1 = a*A + C1



1500×1500 Matrix{Float64}:
  0.694674  -1.7877     0.658208  …   1.00369     3.76447    -1.39682
 -1.88785   -2.28402   -1.67691      -1.22503     1.7104      2.3317
 -2.87171   -0.865292   0.197621     -0.183229   -1.35411     1.49869
 -0.714845  -0.546172   2.77829       0.449245   -0.980041    1.28612
  1.58531    2.1842    -1.79882      -1.58807     2.03121     1.14335
  2.06875    1.27451    0.613698  …  -3.42916    -1.96817     0.0843928
 -0.673988  -0.4472     0.026239      0.585229   -2.79049    -0.324129
 -1.6143     2.63632   -2.21778      -0.616464    2.55836    -1.07941
  1.28246    1.25804    1.51339       1.77383     1.14373     0.339545
 -1.61328   -1.83845   -0.891013      1.6517     -1.43131    -1.37412
  ⋮                               ⋱                          
 -2.12328    1.69287    1.8035       -3.50315     0.443629   -1.99309
  2.54724   -0.748565  -1.6187        2.12674     0.184099    1.85779
  0.699036   0.481519   1.34478      -0.570566   -0.657674   -1.7184

In [ ]:
# Hand AXPY loop 
C2 = similar(A); fill!(C2, 0.0)


function axpy_loop!(C, a, A)
    @inbounds for j in axes(C,2)
        @simd for i in axes(C,1)
            C[i,j] += a*A[i,j]
        end
    end
    return C
end

axpy_loop!(C2, a, A)

@assert isapprox(C1, C2; rtol=1e-12, atol=1e-12)

AssertionError: AssertionError: isapprox(C1, C2; rtol = 1.0e-12, atol = 1.0e-12)

In [ ]:
# Fair, per-trial reset for C1 (broadcast AXPY)
@btime (@. $C1 = $C1 + $a * $A)  setup=(C1 = fill!(similar($A), 0.0));

# Fair, per-trial reset for C2 (your loop AXPY)
@btime axpy_loop!($C2, $a, $A)   setup=(C2 = fill!(similar($A), 0.0));


  1.358 ms (0 allocations: 0 bytes)
  1.018 ms (0 allocations: 0 bytes)


In [1]:
using LinearAlgebra
x = randn(1_000_000); y = randn(1_000_000); α = 2.0
BLAS.axpy!(α, x, y)   # y .= α .* x .+ y  

1000000-element Vector{Float64}:
  1.1944620962830719
  4.246237431315214
  2.310408422613147
  3.1944308273439415
  2.3670040712644114
 -0.2883274325310259
  0.2860492730235437
 -0.06725695176936619
 -0.3031748091617234
 -2.028178784331903
  ⋮
  1.5628097587027159
  0.25269734316878856
 -1.421905705517667
 -1.7264106342260104
  2.04899567340354
  0.54733879327413
  0.04470201077481761
 -1.3557365957645977
  4.127395837682041